In [1]:
import h5py
from model_xray.config import *
from model_xray.path_manager import pm

with h5py.File(pm.get_mcwa_path('famous_le_10m'), 'r') as f:
    for mz_name in f:
        print(mz_name)
        mz = f[mz_name]
        print(mz.dtype, mz.shape)

DenseNet121
float32 (1, 8062504)
EfficientNetV2B0
float32 (1, 7200312)
EfficientNetV2B1
float32 (1, 8212124)
MobileNet
float32 (1, 4253864)
MobileNetV2
float32 (1, 3538984)
MobileNetV3Large
float32 (1, 5507432)
MobileNetV3Small
float32 (1, 2554968)
NASNetMobile
float32 (1, 5326716)


In [3]:
from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import x_lsb_attack, x_lsb_extract

import numpy as np

arr = np.zeros((10, ), dtype=np.uint8)

payload = np.array([0, 1, 2, 3, 4, 5, 6, 7, 20, 14], dtype=np.uint8)
payload_bytes = payload.tobytes()

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload_bytes
)

arr_attacked = x_lsb_attack(arr, config)
print(arr)
print(arr_attacked)

extracted_bytes = x_lsb_extract(arr_attacked, config)

payload_bytes_as_array = np.frombuffer(payload_bytes, dtype=np.uint8)
extracted_bytes_as_array = np.frombuffer(extracted_bytes, dtype=np.uint8)

print(payload_bytes_as_array)
print(extracted_bytes_as_array)

assert extracted_bytes == payload_bytes

capacity: 30, n_b: 8
payload bits: 00000000000000010000001000000011000001000000010100000110000001110001010000001110, len(bits): 80
dupe_amount: 1
len(bits): 80
reshaped: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000']), remainder: Array('bin50', ['11000001000000010100000110000001110001010000001110'])
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 4 0 2 0 0]
capacity: 30, n_b: 8
bits: Array('bin3', ['000', '000', '000', '000', '000', '100', '000', '010', '000', '000'])
[ 0  1  2  3  4  5  6  7 20 14]
[0 1 2 0]


AssertionError: 

In [1]:
import numpy as np
from bitstring import Dtype
import itertools
import os

from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import x_lsb_attack, x_lsb_extract

rng = np.random.default_rng()

dtypes = [np.uint8, np.uint16, np.uint32, np.uint64, np.float32, np.float64]

n_ws = [8, 16, 32, 64]

config = XLSBAttackConfig(
    x = 3,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=None
)

tests_c = 0

for dtype, n_w in itertools.product(dtypes, n_ws):
    dtype_str = dtype.__name__
    # print(dtype_str)
    n_bits = Dtype(dtype_str).length
    # print(n_bits)

    for x in range(1, n_bits):

        capacity = x*n_w
        byte_capacity = capacity // 8

        randbytes = os.urandom(byte_capacity)

        config.x = x
        config.payload_bytes = randbytes

        if np.issubdtype(dtype, np.floating):
            arr = rng.random(size=(n_w, ), dtype=dtype)
        elif np.issubdtype(dtype, np.integer):
            arr = rng.integers(low=0, high = 2**n_bits, size=(n_w,), dtype=dtype)
        else:
            raise ValueError(f"Unknown dtype {dtype}")

        arr_attacked = x_lsb_attack(arr, config)

        config.msb = True
        unattacked_bytes_orig = x_lsb_extract(arr, config)
        unattacked_bytes_attacked = x_lsb_extract(arr_attacked, config)

        assert unattacked_bytes_orig == unattacked_bytes_attacked

        config.msb = False
        extracted_bytes = x_lsb_extract(arr_attacked, config)

        assert extracted_bytes == randbytes
        tests_c += 1

print(f"Tests passed: {tests_c}") 


Tests passed: 840
